<a href="https://colab.research.google.com/github/VarunSaiCSE/CustomChatbot/blob/main/ChatBotCustom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [12]:
import os

#document loader

from langchain_community.document_loaders import PyPDFLoader

#vector store

from langchain_community.vectorstores import Chroma

#LLM

from langchain_openai import OpenAI

In [14]:
loader = PyPDFLoader("/content/aws-overview.pdf")

#upload document as per your need

pages = loader.load()

In [15]:
len(pages)

85

In [16]:
pages[16]

Document(page_content='Overview of Amazon Web Services AWS Whitepaper\nSoftware Development Kits\nSoftware Development Kits\nOur Software Development Kits (SDKs) simplify using AWS services in your applications with an\nApplication Program Interface (API) tailored to your programming language or platform.\nAnalytics\nTopics\n•Amazon Athena (p. 10)\n•Amazon CloudSearch (p. 10)\n•Amazon Elasticsearch Service (p. 11)\n•Amazon EMR (p. 11)\n•Amazon FinSpace (p. 11)\n•Amazon Kinesis (p. 11)\n•Amazon Kinesis Data Firehose (p. 12)\n•Amazon Kinesis Data Analytics (p. 12)\n•Amazon Kinesis Data Streams (p. 12)\n•Amazon Kinesis Video Streams (p. 12)\n•Amazon Redshift (p. 12)\n•Amazon QuickSight (p. 13)\n•AWS Data Exchange (p. 13)\n•AWS Data Pipeline (p. 13)\n•AWS Glue (p. 13)\n•AWS Lake Formation (p. 14)\n•Amazon Managed Streaming for Apache Kafka (Amazon MSK) (p. 14)\nAmazon Athena\nAmazon Athena is an interactive query service that makes it easy to analyze data in Amazon S3 using\nstandard SQL. 

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size = 500, chunk_overlap = 100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
  docs = text_splitter.split_documents(documents)
  return docs


In [18]:
new_pages = split_docs(pages)
len(new_pages)

738

In [19]:
new_pages[500].page_content

'history, and conﬁguration change notiﬁcations to enable security and governance. The Conﬁg Rules\nfeature enables you to create rules that automatically check the conﬁguration of AWS resources recorded\nby AWS Conﬁg.\nWith AWS Conﬁg, you can discover existing and deleted AWS resources, determine your overall\ncompliance against rules, and dive into conﬁguration details of a resource at any point in time. These'

In [20]:
new_pages[499].page_content

'the AWS Management Console, AWS SDKs, command line tools, and higher-level AWS services (such\nas AWS CloudFormation (p. 50)). The AWS API call history produced by CloudTrail enables security\nanalysis, resource change tracking, and compliance auditing.\nAWS Conﬁg\nAWS Conﬁg is a fully managed service that provides you with an AWS resource inventory, conﬁguration\nhistory, and conﬁguration change notiﬁcations to enable security and governance. The Conﬁg Rules'

In [24]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(new_pages, embedding_function)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
from langchain_together import Together

llm = Together(
    model="meta-llama/Llama-2-70b-chat-hf",
    max_tokens=256,
    temperature=0,
    top_k=1,
    together_api_key= "bee087ab937b08c9263017eb303d4107ab23cd1ff24f1975b91003eb6e57d794"

    #https://api.together.ai/settings/api-keys
)

In [37]:
retriever = db.as_retriever(similarity_score_threshold = 0.9)

In [38]:
from langchain.prompts import PromptTemplate
prompt_template = """Please answer questions related to AWS(Cloud services). Try explaining in simple words. Answer in less than 100 words. If you don't know the answer simply respond as "Don't know"
CONTEXT: {context}
QUESTION: {question}"""

PROMPT = PromptTemplate(template = f"[INST] {prompt_template} [/INST]", input_variables=["context", "question"])

In [39]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type='stuff',
    retriever = retriever,
    input_key = 'query',
    return_source_documents = True,
    chain_type_kwargs = {"prompt":PROMPT},
    verbose = True
)

In [41]:
query = input()
response = chain(query)
response['result']

What is AWS?


> Entering new RetrievalQA chain...

> Finished chain.


'  AWS (Amazon Web Services) is a cloud computing platform provided by Amazon that offers a range of services for building, deploying, and managing applications and workloads in the cloud. It provides a highly reliable, scalable, low-cost infrastructure platform that powers hundreds of thousands of businesses in 190 countries around the world. With AWS, users can access a variety of services including computing, storage, databases, security, and management tools, and only pay for what they use.'

In [42]:
query = input()
response = chain(query)
response['result']


amazon redshift per hour fees?


> Entering new RetrievalQA chain...

> Finished chain.


'  Amazon Redshift charges $0.25 per hour with no commitments.'

In [43]:
query = input()
response = chain(query)
response['result']


aws cost


> Entering new RetrievalQA chain...

> Finished chain.


'  AWS cost is a variable expense that is based on consumption. With AWS, you only pay for the computing resources you use, and the amount you consume. This allows you to trade capital expense for variable expense and achieve lower costs compared to on-premises infrastructure. Additionally, AWS provides a pay-as-you-go pricing model that allows you to stop guessing capacity needs and only pay for what you use.'